In [4]:
# --- setup ---
import os, re, glob, pathlib
import pandas as pd
from rapidfuzz import process, fuzz

# Paths (edit if needed)
stata_path = r"C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets\master_dataset_unit_crossection.dta"
excel_path = r"C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\Structural Integrity Reserve Study (SIRS) Reporting(DAwQFJ).xlsx"

excel_dir = r"C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets"

# --- 1) Load master .dta ---
master = pd.read_stata(stata_path, convert_categoricals=False)

# ensure string types we’ll use
for c in ["assoc_name_final","building_address_attom","zip5_attom"]:
    if c in master.columns:
        master[c] = master[c].astype(str).str.strip()
    else:
        raise KeyError(f"Expected column '{c}' not found in master dataset.")

# --- 2) Drop rows where assoc_name_final == building_address_attom ---
master2 = master.loc[master["assoc_name_final"] != master["building_address_attom"]].copy()

# --- 3) Create assoc_name by removing trailing ' ZIP5' (a space + 5 digits) from assoc_name_final ---
master2["assoc_name"] = master2["assoc_name_final"].str.replace(r"\s\d{5}$", "", regex=True).str.strip()

# --- 4) Keep only assoc_name, assoc_name_final, zip5_attom; drop duplicates on (assoc_name, zip5_attom) ---
m_small = master2[["assoc_name","assoc_name_final","zip5_attom"]].dropna(subset=["assoc_name","zip5_attom"]).copy()
m_small = m_small.drop_duplicates(subset=["assoc_name","zip5_attom"])

# --- 5) Load SIRS Excel, keep needed cols, take first 5 chars of Zip, drop duplicates ---
# If the Excel file has multiple sheets and the data is on the first, this works;
# otherwise pass sheet_name="YourSheet"
sirs_raw = pd.read_excel(excel_path, dtype=str)
needed = {"Project Name":"project_name", "Association Name":"association_name", "Zip":"zip"}
missing = [c for c in needed if c not in sirs_raw.columns]
if missing:
    raise KeyError(f"Missing expected Excel columns: {missing}")

sirs = sirs_raw.rename(columns=needed)[["project_name","association_name","zip"]].copy()
sirs["zip5"] = sirs["zip"].astype(str).str.strip().str[:5]
sirs = sirs.drop(columns=["zip"]).dropna(subset=["zip5"])
sirs = sirs.drop_duplicates(subset=["project_name","association_name","zip5"]).reset_index(drop=True)

# Duplicate the two SIRS name columns into a single long “name” field so either can match
sirs_long = pd.DataFrame({
    "sirs_name_raw": pd.concat([sirs["project_name"], sirs["association_name"]], ignore_index=True),
    "zip5":          pd.concat([sirs["zip5"],         sirs["zip5"]],          ignore_index=True),
    "sirs_source":   ["PROJECT_NAME"]*len(sirs) + ["ASSOCIATION_NAME"]*len(sirs)
}).dropna(subset=["sirs_name_raw","zip5"]).drop_duplicates().reset_index(drop=True)

# --- 6) Standardization function (uppercase, suffix INC removal, standardize tokens, rm punctuation) ---
INC_SUFFIX_RE = re.compile(r"(,?\s+INC\.?)\s*$")
def std_name(x: str) -> str:
    if pd.isna(x): return x
    s = str(x).upper().strip()

    # remove trailing INC variants at end only
    s = INC_SUFFIX_RE.sub("", s)

    # standardize tokens
    # typos to CONDO
    s = re.sub(r"\bCONDOMIUM\b|\bCONDMINIUM\b|\bCONDOMINIUM\b", "CONDO", s)
    # association family -> ASSOC
    s = re.sub(r"\bASSOCIATION\b|\bASSN\.?\b|\bASSOC\.?\b", "ASSOC", s)
    # apartments -> APTS
    s = re.sub(r"\bAPARTMENTS\b", "APTS", s)

    # remove commas and periods
    s = s.replace(",", "").replace(".", "")

    # collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()
    return s

m_small["assoc_name_std"] = m_small["assoc_name"].map(std_name)
sirs_long["sirs_name_std"] = sirs_long["sirs_name_raw"].map(std_name)

# also align ZIPs to strings & consistent names
m_small = m_small.rename(columns={"zip5_attom":"zip5"})
m_small["zip5"] = m_small["zip5"].astype(str).str.strip().str[:5]

# --- 7) Fuzzy match within ZIP (>=90), using RapidFuzz token_set_ratio ---
def match_within_zip(zip_code, sirs_block, master_block):
    # build choices dict {name_std: (assoc_name_std, assoc_name, assoc_name_final)}
    choices = dict()
    for _, r in master_block.iterrows():
        choices[r["assoc_name_std"]] = (r["assoc_name_std"], r["assoc_name"], r["assoc_name_final"])

    out = []
    for _, r in sirs_block.iterrows():
        q = r["sirs_name_std"]
        if not q or not len(choices):
            continue
        match = process.extractOne(
            q,
            choices.keys(),
            scorer=fuzz.token_set_ratio,
            score_cutoff=90  # 90% threshold
        )
        if match:
            key, score, _ = match
            assoc_name_std, assoc_name, assoc_name_final = choices[key]
            out.append({
                "zip5": zip_code,
                "sirs_name_raw": r["sirs_name_raw"],
                "sirs_name_std": q,
                "sirs_source": r["sirs_source"],
                "assoc_name_std": assoc_name_std,
                "assoc_name": assoc_name,
                "assoc_name_final": assoc_name_final,
                "match_score": score
            })
    return out

matches = []
for z, sblk in sirs_long.groupby("zip5", sort=False):
    mblk = m_small.loc[m_small["zip5"]==z]
    if len(mblk)==0: 
        continue
    matches.extend(match_within_zip(z, sblk, mblk))

crosswalk = pd.DataFrame(matches).drop_duplicates()

# --- 8) Save crosswalk in the same folder ---
crosswalk_path = os.path.join(excel_dir, "sirs_assoc_crosswalk_by_zip_fuzzy90.csv")
crosswalk.to_csv(crosswalk_path, index=False)

# --- 9) Add SIRS_self_report to the ORIGINAL master without altering anything else ---
# A pair is “covered” if its (assoc_name_final, zip5_attom) appears in matched results
matched_keys = set(
    crosswalk[["assoc_name_final","zip5"]]
    .dropna()
    .drop_duplicates()
    .itertuples(index=False, name=None)
)

# create the flag without touching other columns
master_out = master.copy()
master_out["zip5_attom"] = master_out["zip5_attom"].astype(str).str.strip().str[:5]
master_out["SIRS_self_report"] = master_out[["assoc_name_final","zip5_attom"]].apply(
    lambda r: 1 if (str(r["assoc_name_final"]).strip(), str(r["zip5_attom"]).strip()) in matched_keys else 0,
    axis=1
).astype("int8")

# --- 10) Save master with new flag (write a new file to avoid overwriting) ---
out_path = os.path.join(
    os.path.dirname(stata_path),
    pathlib.Path(stata_path).stem + "_with_SIRS_self_report.dta"
)
master_out.to_stata(out_path, write_index=False)

print("Crosswalk saved to:", crosswalk_path)
print("Updated master saved to:", out_path)


Crosswalk saved to: C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets\sirs_assoc_crosswalk_by_zip_fuzzy90.csv
Updated master saved to: C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets\master_dataset_unit_crossection_with_SIRS_self_report.dta


In [5]:
import os
import pandas as pd

# ---- paths ----
unit_path = r"C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets\master_dataset_unit_crossection_with_SIRS_self_report.dta"

# ---- load + keep needed cols ----
use_cols = ["assoc_name_final", "SIRS_self_report", "mm_fips_county_name_attom", "treated_assoc"]
df = pd.read_stata(unit_path, columns=use_cols, convert_categoricals=False)

# ensure expected dtypes
df["assoc_name_final"] = df["assoc_name_final"].astype(str).str.strip()
df["mm_fips_county_name_attom"] = df["mm_fips_county_name_attom"].astype(str).str.strip()
df["SIRS_self_report"] = pd.to_numeric(df["SIRS_self_report"], errors="coerce").fillna(0).astype(int)
df["treated_assoc"] = pd.to_numeric(df["treated_assoc"], errors="coerce").fillna(0).astype(int)

# ---- filter to treated (3+ stories) and dedupe by association name ----
treated = (
    df.loc[df["treated_assoc"] == 1, ["assoc_name_final", "SIRS_self_report", "mm_fips_county_name_attom"]]
      .drop_duplicates(subset=["assoc_name_final"])
      .rename(columns={"mm_fips_county_name_attom": "county"})
)

# ---- by-county counts and fractions ----
by_cty = (
    treated.groupby("county", dropna=False)
           .agg(total_3plus_assocs=("assoc_name_final", "count"),
                sirs_report_count=("SIRS_self_report", "sum"))
           .reset_index()
)
by_cty["frac_sirs"] = by_cty["sirs_report_count"] / by_cty["total_3plus_assocs"]

# ---- overall row ----
overall = pd.DataFrame({
    "county": ["OVERALL"],
    "total_3plus_assocs": [len(treated)],
    "sirs_report_count": [treated["SIRS_self_report"].sum()]
})
overall["frac_sirs"] = overall["sirs_report_count"] / overall["total_3plus_assocs"]

# ---- final table (by county + overall), sorted by fraction desc ----
table = pd.concat([by_cty, overall], ignore_index=True)
table = table.sort_values(
    by=["county"], key=lambda s: s.where(s.eq("OVERALL"), "A"+s)  # keep OVERALL at bottom
)

# pretty percentage column if you want to display/export
table_display = table.copy()
table_display["frac_sirs_pct"] = (table_display["frac_sirs"] * 100).round(1)

# --- sort by total_3plus_assocs (largest -> smallest) and print ---
sorted_table = (
    table_display.loc[table_display["county"] != "OVERALL"]  # drop overall for sorting
                 .sort_values("total_3plus_assocs", ascending=False)
                 .reset_index(drop=True)
)
print(sorted_table)  # print full sorted table if you want
print("\nTop 10 by total 3+ story associations:\n")
print(sorted_table.head(10))

          county  total_3plus_assocs  sirs_report_count  frac_sirs  \
0     MIAMI-DADE                1821                492   0.270181   
1        BROWARD                1533                467   0.304631   
2       PINELLAS                 867                217   0.250288   
3     PALM BEACH                 782                232   0.296675   
4            LEE                 390                 89   0.228205   
5       SARASOTA                 340                 87   0.255882   
6        COLLIER                 334                 81   0.242515   
7        VOLUSIA                 244                 79   0.323770   
8        BREVARD                 190                 54   0.284211   
9         ORANGE                 185                 20   0.108108   
10       MANATEE                 181                 46   0.254144   
11  HILLSBOROUGH                 171                 27   0.157895   
12      SEMINOLE                 163                  8   0.049080   
13     ST. JOHNS    

In [6]:
# --- take top 10 and build LaTeX table ---
top10 = sorted_table.head(10).copy()

# keep county, total, and the percent column (not the raw fraction)
top10 = top10[["county", "total_3plus_assocs", "frac_sirs_pct"]].rename(
    columns={
        "county": "County",
        "total_3plus_assocs": "3+ Story Associations",
        "frac_sirs_pct": "Self-reported SIRS (%)"
    }
)

# format percent with a trailing % for LaTeX
top10["Self-reported SIRS (%)"] = top10["Self-reported SIRS (%)"].map(lambda x: f"{x:.1f}\\%")

# export to LaTeX in the same folder as the unit file
latex_out = os.path.join(os.path.dirname(unit_path), "top10_sirs_by_county.tex")
top10.to_latex(
    latex_out,
    index=False,
    escape=False,                 # allow the % signs we added
    column_format="lrr",          # left, right, right
    caption="Top 10 Florida counties by number of 3+ story condo associations and their self-reported SIRS rates.",
    label="tab:top10_sirs_by_county"
)

print("LaTeX table saved to:", latex_out)

LaTeX table saved to: C:\Users\ngodin\Dropbox\RESEARCH\active_projects\florida_condo\final_datasets\master_datasets\top10_sirs_by_county.tex


C:\Users\ngodin\AppData\Local\Temp\ipykernel_29928\3144523900.py:18: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  top10.to_latex(
